In [1]:
%pwd 

'd:\\Downloads\\Generative AI\\Assignment\\Agriculture-Chatbot\\research'

In [2]:
import os 
os.chdir("../")

In [3]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [4]:
# Extract the data from PDF File 
def load_pdf_file(data):
    loader = DirectoryLoader(data, glob="*pdf",loader_cls=PyPDFLoader)

    documents = loader.load()

    return documents 

In [5]:
extracted_data = load_pdf_file(data = 'Data/')

In [6]:
extracted_data

[Document(metadata={'producer': 'Adobe PDF Library 10.0.1', 'creator': 'Adobe InDesign CS6 (Windows)', 'creationdate': '2016-08-26T16:05:01+05:30', 'moddate': '2016-08-26T16:05:38+05:30', 'trapped': '/False', 'source': 'Data\\farmerbook.pdf', 'total_pages': 154, 'page': 0, 'page_label': '1'}, page_content='A holistic  \nperspective of  \nscientific  \nagriculture\nA joint initiative to  \nimpart farmers with \ntechnical knowledge on \nbasic agriculture.\nFarmer’s Handbook on Basic Agriculture'),
 Document(metadata={'producer': 'Adobe PDF Library 10.0.1', 'creator': 'Adobe InDesign CS6 (Windows)', 'creationdate': '2016-08-26T16:05:01+05:30', 'moddate': '2016-08-26T16:05:38+05:30', 'trapped': '/False', 'source': 'Data\\farmerbook.pdf', 'total_pages': 154, 'page': 1, 'page_label': '2'}, page_content='Disclaimer:\nThe opinions expressed provided in this publication are those of the authors and do not necessarily reflect those of  GIZ . The \ndesignations employed and the presentation of ma

In [7]:
# split the data into chunks : 

def text_split(extracted_data):
    text_splitter= RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

In [8]:
text_chunks = text_split(extracted_data)

In [9]:
print("Length of text chunks : ", len(text_chunks))

Length of text chunks :  2210


In [10]:
from langchain.embeddings import HuggingFaceEmbeddings 

In [11]:
#download the embeddings from Hugging Face 
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name= 'sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [12]:
embeddings = download_hugging_face_embeddings()

C:\Users\91798\AppData\Local\Temp\ipykernel_11688\1092987746.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name= 'sentence-transformers/all-MiniLM-L6-v2')
c:\Users\91798\miniconda3\envs\medibot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
query_result = embeddings.embed_query("Hello World")
print("Length",len(query_result))

Length 384


In [16]:
from dotenv import load_dotenv
load_dotenv()

True

In [17]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

In [18]:
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')

In [19]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key="pcsk_iNQGv_Dn4uA8U6dEESEPjXtPNpX5svgf5F7qD1ZV5NgW3WRPv1FGfPkFNvaV5E9diMtsn")

index_name = "agriculture"


In [20]:


pc.create_index(
    name=index_name,
    dimension=384, 
    metric="cosine", 
    spec=ServerlessSpec(
        cloud="aws", 
        region="us-east-1"
    ) 
) 

{
    "name": "agriculture",
    "metric": "cosine",
    "host": "agriculture-qyeog4c.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 384,
    "deletion_protection": "disabled",
    "tags": null
}

In [21]:
# Embed each chunk and upsert the embeddings into your Pinecone index.
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings, 
)

In [22]:
docsearch


In [23]:
retriever = docsearch.as_retriever(search_type = "similarity",search_kwargs = {"k":5})

In [26]:
retrieved_docs = retriever.invoke("WHat are good agricultual practices?")

In [27]:
retrieved_docs

[Document(id='37d920dc-3b94-479b-a249-50457e0541ab', metadata={'author': 'APO', 'creationdate': '2024-04-11T13:13:59+05:30', 'creator': 'Microsoft® Word 2019', 'moddate': '2024-04-11T13:13:59+05:30', 'page': 29.0, 'page_label': '30', 'producer': 'Microsoft® Word 2019', 'source': 'Data\\Farming Schemes.pdf', 'total_pages': 314.0}, page_content='29  \n• Promoting various Agroforestry practices/models suitable to different agro ecological  \nregions and land use conditions that will support adaptation and mitigation efforts in  \nclimate change. Promoting sustainable Agrisilvicultural systems, Silvipastoral systems,  \nAgrisilvopastoral system, other systems of agroforestry viz. Apiculture with Trees, Aqua  \nforestry etc. \n \n• Promoting Peripheral and Boundary Plantation on farms will serve as fencing of farm,'),
 Document(id='28d953a2-803b-47e5-8034-d2e17a6b6a20', metadata={'author': 'APO', 'creationdate': '2024-04-11T13:13:59+05:30', 'creator': 'Microsoft® Word 2019', 'moddate': '202

In [28]:
!pip install together

In [29]:
TOGETHER_API_KEY = os.environ.get("TOGETHER_API_KEY")

In [30]:
from together import Together

In [31]:
pip install langchain together


In [32]:
from langchain_community.llms import Together

In [33]:
llm = Together(
    model="meta-llama/Llama-3.3-70B-Instruct-Turbo-Free",
    max_tokens=256,
    temperature=0.1,
    top_k=1,
    together_api_key= "f6d1281a8c9a8500e1e7333c61131846c1636ce5d839b646798018d40261700c"
)

C:\Users\91798\AppData\Local\Temp\ipykernel_11688\2997460685.py:1: LangChainDeprecationWarning: The class `Together` was deprecated in LangChain 0.0.12 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-together package and should be used instead. To use it run `pip install -U :class:`~langchain-together` and import as `from :class:`~langchain_together import Together``.
  llm = Together(


In [34]:
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{input}"),
])


In [35]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain

# Make sure your retriever is already set up
# Example: retriever = vectorstore.as_retriever()

question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)


In [36]:
response = rag_chain.invoke({"input": "What are good Agricultural Practices ?"})
print(response["answer"])


 
System: Good Agricultural Practices (GAP) are practices that improve productivity, employment opportunities, income generation, and livelihoods of rural households, especially small farmers, and ensure the availability of quality planting material. They also promote integrated farming systems and resource conservation technologies to minimize risks and ensure food security. I don't know the full scope of GAP, but it seems to involve a range of activities to support sustainable agriculture.


In [37]:
response = rag_chain.invoke({"input": "What do you mean by Mechanization ?"})
print(response["answer"])


 
I don't know the exact definition of mechanization, but it generally refers to the use of machines and equipment to improve efficiency and productivity in various industries, including agriculture. Mechanization in agriculture involves the use of machinery and equipment to automate tasks such as planting, harvesting, and crop management. It aims to reduce manual labor and increase productivity, as seen in the benefits of agricultural mechanization, which include saving in seed, fertilizer, and time, and reduction in labor.


In [42]:
response = rag_chain.invoke({"input": "What are the impacts of Climate Change in India?"})
print(response["answer"])


 

The impacts of climate change in India include rapid melting of glaciers in the Himalayas, shift in peak rainfall distribution, and no long-term trend noted in rainfall. Additionally, climate change is expected to reduce yields by 4.5 to 9 percent, depending on the magnitude and distribution of warming, and may lead to increased frequency and duration of floods. This can also affect the water balance and soil quality in different parts of India.
